In [327]:
import cartolafc
import pandas as pd
import json
import time
import requests
from functools import lru_cache

pd.set_option('display.max_columns', 50)            # permite a visualizacao de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os numeros aparecerem com duas casas decimais

# Cria uma instancia da API
api = cartolafc.Api(attempts=5)

# Constantes do 1? turno
INICIO_TURNO = 1
FIM_TURNO = 19
COLUNAS_RODADAS = [f"Rodada {r}" for r in range(INICIO_TURNO, FIM_TURNO + 1)]


19209079, 1488983, 287965, 2916559, 186283, 2371918, 16411206, 14933455, 47775950', 1747619, 32966, 44810918, 1867254, 4088673, 20651178, 14709358, 184499, 1273719

In [328]:
# Lista fixa de IDs dos participantes (lista de int)
ids_participantes = [ 
                      19209079, 1488983, 287965, 2916559, 186283, 2371918, 16411206, 14933455, 47775950, 1747619,
                      32966, 44810918, 1867254, 4088673, 20651178, 14709358, 184499, 1273719, 1326835, 19833277
                    ]

In [329]:
HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json,text/plain,*/*",
    "Referer": "https://cartola.globo.com/",
}

@lru_cache(maxsize=5000)
def nome_time_por_id_api(time_id: int, timeout=15) -> str:
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id}",
        f"https://api.cartolafc.globo.com/time/{time_id}",
    ]

    for url in endpoints:
        for tentativa in range(3):
            try:
                r = requests.get(url, headers=HEADERS, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(0.8 + tentativa * 0.8)
                    continue
                if r.status_code != 200:
                    break

                data = r.json()
                if isinstance(data, dict):
                    if isinstance(data.get("time"), dict) and isinstance(data["time"].get("nome"), str):
                        return data["time"]["nome"]
                    if isinstance(data.get("nome"), str):
                        return data["nome"]
                break
            except Exception:
                time.sleep(0.5)
                continue

    return f"Time {time_id}"


In [330]:
# Base com todos os participantes
if not isinstance(ids_participantes, list) or not ids_participantes:
    raise ValueError("ids_participantes precisa ser uma lista de IDs")

ids_participantes = list(dict.fromkeys(ids_participantes))

df_base = pd.DataFrame({"time_id": ids_participantes}).drop_duplicates()
df_base["Time"] = df_base["time_id"].apply(nome_time_por_id_api)

df_base = df_base.set_index("time_id").sort_index()

# Dicionario Nome -> ID (compatibilidade com codigo legado)
ids_times_dict = {row["Time"]: row["time_id"] for _, row in df_base.reset_index().iterrows()}

# Links para o Excel
df_urls = pd.DataFrame({
    "Nome do Time": df_base["Time"].values,
    "ID do Time": df_base.index.values,
})

df_urls["Link do Time"] = df_urls["ID do Time"].apply(
    lambda x: f"https://cartola.globo.com/#!/time/{x}"
)

df_urls = df_urls[["Nome do Time", "ID do Time", "Link do Time"]]

caminho_excel = "links_times_cartola_liga_eliminacao_20.xlsx"
df_urls.to_excel(caminho_excel, index=False)
print(f"? Arquivo salvo com sucesso: {caminho_excel}")

display(df_base)
display(df_urls)


? Arquivo salvo com sucesso: links_times_cartola_liga_eliminacao_20.xlsx


,Time
time_id,
32966,La Primeira Patada Es Nuestra
184499,SC ÉoINTER!
186283,FBC Colorado
287965,Doug Leal F.C
1273719,Texas Club 2026
1326835,Pepe Leal FC
1488983,C R Juvenal
1747619,JV5 Tricolor Gaúcho
1867254,Medonho´s F.C.


,Nome do Time,ID do Time,Link do Time
0,La Primeira Patada Es Nuestra,32966,https://cartola.globo.com/#!/time/32966
1,SC ÉoINTER!,184499,https://cartola.globo.com/#!/time/184499
2,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
3,Doug Leal F.C,287965,https://cartola.globo.com/#!/time/287965
4,Texas Club 2026,1273719,https://cartola.globo.com/#!/time/1273719
5,Pepe Leal FC,1326835,https://cartola.globo.com/#!/time/1326835
6,C R Juvenal,1488983,https://cartola.globo.com/#!/time/1488983
7,JV5 Tricolor Gaúcho,1747619,https://cartola.globo.com/#!/time/1747619
8,Medonho´s F.C.,1867254,https://cartola.globo.com/#!/time/1867254
9,GaúchoDaFronteira F.C,2371918,https://cartola.globo.com/#!/time/2371918


In [331]:
def campeonato_comecou(ids, rodada_ref=INICIO_TURNO):
    lista_ids = list(ids.values()) if isinstance(ids, dict) else list(ids)
    for time_id in lista_ids:
        try:
            t = api.time(time_id=time_id, rodada=rodada_ref)
            v = getattr(t, "ultima_pontuacao", None)
            if v is not None:
                return True
        except Exception:
            continue
    return False


In [332]:
def safe_get_points(time_id, rodada):
    try:
        t = api.time(time_id=time_id, rodada=rodada)
        for attr in ("pontos", "pontuacao", "ultima_pontuacao"):
            v = getattr(t, attr, None)
            if v is not None:
                return float(v)
        return 0.0
    except Exception:
        return 0.0

# Rodada atual limitada ao 1? turno
try:
    rodada_atual = min(api.mercado().rodada_atual, FIM_TURNO)
except Exception:
    rodada_atual = FIM_TURNO

rodadas_ate_atual = list(range(INICIO_TURNO, rodada_atual + 1))

df_pontuacoes = df_base.copy()
for col in COLUNAS_RODADAS:
    df_pontuacoes[col] = ""

eliminados_por_rodada = {}

times_ativos = list(df_pontuacoes.index)

if not campeonato_comecou(ids_participantes, rodada_ref=INICIO_TURNO):
    print("? O campeonato ainda nao comecou. Criando tabela mock com Rodada 1 = 0.0.")
    df_pontuacoes["Rodada 1"] = 0.0
else:
    # Logica de eliminacao do 1? turno
    limite_um = INICIO_TURNO + 8  # Rodadas 1..9 eliminam 1

    for r in rodadas_ate_atual:
        print(f"? Rodada {r} ? Times participantes: {len(times_ativos)}")

        for time_id in df_pontuacoes.index:
            if time_id in times_ativos:
                pontos = safe_get_points(time_id, r)
                df_pontuacoes.at[time_id, f"Rodada {r}"] = pontos
            else:
                df_pontuacoes.at[time_id, f"Rodada {r}"] = ""

        if r < FIM_TURNO:
            n_elimina = 1 if r <= limite_um else 2
            if len(times_ativos) > 5:
                ativos_df = df_pontuacoes.loc[times_ativos].copy()
                ativos_df["pontos_rodada"] = pd.to_numeric(ativos_df[f"Rodada {r}"], errors="coerce").fillna(0.0)
                piores = ativos_df.sort_values("pontos_rodada", ascending=True).head(n_elimina)
                ids_eliminados = piores.index.tolist()
                times_ativos = [x for x in times_ativos if x not in ids_eliminados]
                eliminados_por_rodada[r] = ids_eliminados

# Exibir DataFrame final
# (mantem todos os times, mesmo sem pontuacao real)
display(df_pontuacoes)


2026-01-29 12:20:14,958 - root - ERROR - Este time ainda não foi escalado na temporada.


? O campeonato ainda nao comecou. Criando tabela mock com Rodada 1 = 0.0.


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19
time_id,,,,,,,,,,,,,,,,,,,,
32966,La Primeira Patada Es Nuestra,0.00,,,,,,,,,,,,,,,,,,
184499,SC ÉoINTER!,0.00,,,,,,,,,,,,,,,,,,
186283,FBC Colorado,0.00,,,,,,,,,,,,,,,,,,
287965,Doug Leal F.C,0.00,,,,,,,,,,,,,,,,,,
1273719,Texas Club 2026,0.00,,,,,,,,,,,,,,,,,,
1326835,Pepe Leal FC,0.00,,,,,,,,,,,,,,,,,,
1488983,C R Juvenal,0.00,,,,,,,,,,,,,,,,,,
1747619,JV5 Tricolor Gaúcho,0.00,,,,,,,,,,,,,,,,,,
1867254,Medonho´s F.C.,0.00,,,,,,,,,,,,,,,,,,


In [333]:
# ================================
# Pontuacao parcial (rodada em andamento)
# ================================

def fetch_pontuados(timeout=15):
    url = "https://api.cartola.globo.com/atletas/pontuados"
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        if r.status_code != 200:
            return {}
        data = r.json()
        atletas = data.get("atletas", {}) if isinstance(data, dict) else {}
        if not isinstance(atletas, dict):
            return {}
        out = {}
        for k, v in atletas.items():
            try:
                atleta_id = int(k)
                pont = v.get("pontuacao") if isinstance(v, dict) else None
                if pont is None:
                    continue
                out[atleta_id] = float(pont)
            except Exception:
                continue
        return out
    except Exception:
        return {}


def fetch_time_payload(time_id: int, rodada: int, timeout=15):
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id}/{rodada}",
        f"https://api.cartolafc.globo.com/time/{time_id}/{rodada}",
    ]

    for url in endpoints:
        for tentativa in range(3):
            try:
                r = requests.get(url, headers=HEADERS, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(0.8 + tentativa * 0.8)
                    continue
                if r.status_code != 200:
                    break
                data = r.json()
                if not isinstance(data, dict):
                    data = {}
                return data
            except Exception:
                time.sleep(0.5)
                continue
    return {}


def fetch_partidas_rodada(rodada: int, timeout=15):
    url = f"https://api.cartolafc.globo.com/partidas/{rodada}"
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        if r.status_code != 200:
            return []
        data = r.json()
        if isinstance(data, dict):
            partidas = data.get("partidas", [])
            return partidas if isinstance(partidas, list) else []
    except Exception:
        pass
    return []


def clubes_que_ja_jogaram(rodada: int):
    partidas = fetch_partidas_rodada(rodada)
    clubes = set()
    for p in partidas:
        if not isinstance(p, dict):
            continue
        status = (p.get("status_transmissao_tr") or p.get("status_transmissao") or "").upper()
        encerrada = status in ("ENCERRADA", "FINALIZADA")
        if p.get("placar_oficial_mandante") is not None or p.get("placar_oficial_visitante") is not None:
            encerrada = True
        if encerrada:
            mid = p.get("clube_casa_id")
            vid = p.get("clube_visitante_id")
            try:
                if mid is not None:
                    clubes.add(int(mid))
            except Exception:
                pass
            try:
                if vid is not None:
                    clubes.add(int(vid))
            except Exception:
                pass
    return clubes


def setor_por_posicao(posicao_id: int) -> str:
    mapa = {
        1: "Goleiro",
        2: "Laterais",
        3: "Zagueiros",
        4: "Meias",
        5: "Atacantes",
        6: "Tecnico",
    }
    return mapa.get(int(posicao_id), "")


def _id_int(val):
    try:
        return int(val)
    except Exception:
        return None


def calcular_parcial_time(time_id: int, rodada: int, mapa_pontuados: dict, clubes_jogaram: set):
    data = fetch_time_payload(time_id, rodada)
    atletas = data.get("atletas") if isinstance(data, dict) else None
    if not isinstance(atletas, list):
        return 0.0, [], None

    titulares = []
    tecnico = None
    for a in atletas:
        if not isinstance(a, dict):
            continue
        pos = a.get("posicao_id")
        if pos == 6:
            tecnico = a
        else:
            titulares.append(a)

    luxo_id = data.get("reserva_luxo_id") if isinstance(data, dict) else None
    if luxo_id is not None:
        luxo_id = _id_int(luxo_id)

    # Capitao (1.5x)
    capitao_id = None
    try:
        capitao_id = data.get("capitao_id") if isinstance(data, dict) else None
        if capitao_id is None and isinstance(data.get("time"), dict):
            capitao_id = data["time"].get("capitao_id")
        capitao_id = _id_int(capitao_id)
    except Exception:
        capitao_id = None

    titulares_por_pos = {}
    for a in titulares:
        pos = a.get("posicao_id")
        titulares_por_pos.setdefault(pos, []).append(a)

    atletas_em_jogo = list(titulares)
    subs_banco = []

    reservas = data.get("reservas", []) if isinstance(data, dict) else []
    if isinstance(reservas, list):
        for r in reservas:
            if not isinstance(r, dict):
                continue
            # reserva de luxo nao entra como banco
            if luxo_id is not None and _id_int(r.get("atleta_id")) == luxo_id:
                continue
            # banco so entra se reserva tiver pontuacao parcial positiva
            rid = _id_int(r.get("atleta_id"))
            if rid is None or rid not in mapa_pontuados:
                continue
            if float(mapa_pontuados.get(rid, 0.0)) <= 0.0:
                continue
            pos = r.get("posicao_id")
            if pos is None:
                continue
            candidatos = titulares_por_pos.get(pos, [])
            titular_sub = None
            for t in candidatos:
                tid = _id_int(t.get("atleta_id"))
                if tid is None:
                    continue
                clube_tid = _id_int(t.get("clube_id"))
                if tid not in mapa_pontuados and (clube_tid in clubes_jogaram):
                    titular_sub = t
                    break
            if titular_sub is None:
                continue
            try:
                atletas_em_jogo.remove(titular_sub)
            except ValueError:
                pass
            atletas_em_jogo.append(r)
            subs_banco.append((titular_sub.get("atleta_id"), r.get("atleta_id"), pos))

    # Reserva de luxo
    sub_luxo = None
    luxo_capitao = False
    if luxo_id is not None:
        ids_em_jogo = set()
        for a in atletas_em_jogo:
            ids_em_jogo.add(_id_int(a.get("atleta_id")))
        if luxo_id not in ids_em_jogo:
            luxo_obj = None
            if isinstance(reservas, list):
                for r in reservas:
                    if isinstance(r, dict) and _id_int(r.get("atleta_id")) == luxo_id:
                        luxo_obj = r
                        break
            if isinstance(luxo_obj, dict):
                p_luxo = float(mapa_pontuados.get(luxo_id, 0.0))
                if p_luxo > 0:
                    setor_luxo = setor_por_posicao(luxo_obj.get("posicao_id"))
                    candidatos_setor = []
                    for a in atletas_em_jogo:
                        if setor_por_posicao(a.get("posicao_id")) == setor_luxo:
                            candidatos_setor.append(a)
                    if candidatos_setor:
                        # luxo so entra se TODOS do setor ja jogaram
                        todos_setor_jogaram = True
                        for c in candidatos_setor:
                            clube_c = _id_int(c.get("clube_id"))
                            if clube_c not in clubes_jogaram:
                                todos_setor_jogaram = False
                                break
                        if todos_setor_jogaram:
                            def pts(a):
                                try:
                                    return float(mapa_pontuados.get(int(a.get("atleta_id")), 0.0))
                                except Exception:
                                    return 0.0
                            # pior titular do setor; desempate favorece capit?o
                            pior_pts = None
                            piores = []
                            for c in candidatos_setor:
                                v = pts(c)
                                if pior_pts is None or v < pior_pts:
                                    pior_pts = v
                                    piores = [c]
                                elif v == pior_pts:
                                    piores.append(c)
                            pior = None
                            if len(piores) > 1 and capitao_id is not None:
                                for c in piores:
                                    if _id_int(c.get("atleta_id")) == capitao_id:
                                        pior = c
                                        break
                            if pior is None:
                                pior = piores[0] if piores else None
                            if pior is not None and p_luxo > pts(pior):
                                if capitao_id is not None and _id_int(pior.get("atleta_id")) == capitao_id:
                                    luxo_capitao = True
                                try:
                                    atletas_em_jogo.remove(pior)
                                except ValueError:
                                    pass
                                atletas_em_jogo.append(luxo_obj)
                                sub_luxo = (pior.get("atleta_id"), luxo_id, setor_luxo)

    total = 0.0
    for a in atletas_em_jogo:
        try:
            aid = int(a.get("atleta_id"))
        except Exception:
            continue
        total += float(mapa_pontuados.get(aid, 0.0))

    # Bonus do capitao (50%)
    if capitao_id is not None:
        cap_pts = float(mapa_pontuados.get(capitao_id, 0.0))
        if cap_pts != 0.0:
            ids_em_jogo = set()
            for a in atletas_em_jogo:
                ids_em_jogo.add(_id_int(a.get("atleta_id")))
            if capitao_id in ids_em_jogo:
                total += cap_pts * 0.5
            elif luxo_capitao and luxo_id is not None and luxo_id in ids_em_jogo:
                luxo_pts = float(mapa_pontuados.get(luxo_id, 0.0))
                if luxo_pts != 0.0:
                    total += luxo_pts * 0.5

    if isinstance(tecnico, dict):
        try:
            tid = int(tecnico.get("atleta_id"))
            total += float(mapa_pontuados.get(tid, 0.0))
        except Exception:
            pass

    return round(total, 2), subs_banco, sub_luxo


# Detecta se ha parcial disponivel
clubes_jogaram = clubes_que_ja_jogaram(rodada_atual)
mapa_pontuados = fetch_pontuados()
col_parcial = f"Parcial Rodada {rodada_atual}"

if col_parcial not in df_pontuacoes.columns:
    df_pontuacoes[col_parcial] = ""

_debug_exemplo = None

if mapa_pontuados:
    for time_id in df_pontuacoes.index:
        parcial, subs_banco, sub_luxo = calcular_parcial_time(int(time_id), rodada_atual, mapa_pontuados, clubes_jogaram)
        df_pontuacoes.at[time_id, col_parcial] = parcial
        if _debug_exemplo is None:
            _debug_exemplo = (int(time_id), subs_banco, sub_luxo)

    df_parcial_rodada_atual = df_pontuacoes[["Time", col_parcial]].copy()
    df_parcial_rodada_atual = df_parcial_rodada_atual.rename(columns={col_parcial: "Pontuacao_Parcial"})
    df_parcial_rodada_atual["Rodada"] = int(rodada_atual)
else:
    df_parcial_rodada_atual = df_pontuacoes[["Time"]].copy()
    df_parcial_rodada_atual["Rodada"] = int(rodada_atual)
    df_parcial_rodada_atual["Pontuacao_Parcial"] = ""

# Exibe parcial
try:
    display(df_parcial_rodada_atual.sort_values("Pontuacao_Parcial", ascending=False))
except Exception:
    display(df_parcial_rodada_atual)

# Debug curto para 1 time
if _debug_exemplo is not None:
    tid, sb, sl = _debug_exemplo
    nome = df_pontuacoes.at[tid, "Time"] if tid in df_pontuacoes.index else tid
    total_parcial = df_pontuacoes.at[tid, col_parcial]
    print("DEBUG TIME:", tid, "-", nome)
    print("TOTAL PARCIAL:", total_parcial)
    print("SUB BANCO:", sb[:5])
    print("SUB LUXO:", sl)

# Mantem exibicao do df principal
try:
    display(df_pontuacoes)
except Exception:
    pass


,Time,Pontuacao_Parcial,Rodada
time_id,,,
2916559,Esquadrão Gazembrino,83.50,1
14709358,SC 100 Sono,67.16,1
2371918,GaúchoDaFronteira F.C,63.76,1
19209079,bugredasmissões,62.76,1
1273719,Texas Club 2026,59.86,1
287965,Doug Leal F.C,58.17,1
1867254,Medonho´s F.C.,55.96,1
44810918,lsauer fc,54.36,1
47775950,Grêmio_Campeão_LA_27,53.91,1


DEBUG TIME: 32966 - La Primeira Patada Es Nuestra
TOTAL PARCIAL: 44.26
SUB BANCO: []
SUB LUXO: (132071, 92696, 'Atacantes')


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19,Parcial Rodada 1
time_id,,,,,,,,,,,,,,,,,,,,,
32966,La Primeira Patada Es Nuestra,0.00,,,,,,,,,,,,,,,,,,,44.26
184499,SC ÉoINTER!,0.00,,,,,,,,,,,,,,,,,,,40.40
186283,FBC Colorado,0.00,,,,,,,,,,,,,,,,,,,45.46
287965,Doug Leal F.C,0.00,,,,,,,,,,,,,,,,,,,58.17
1273719,Texas Club 2026,0.00,,,,,,,,,,,,,,,,,,,59.86
1326835,Pepe Leal FC,0.00,,,,,,,,,,,,,,,,,,,16.40
1488983,C R Juvenal,0.00,,,,,,,,,,,,,,,,,,,38.46
1747619,JV5 Tricolor Gaúcho,0.00,,,,,,,,,,,,,,,,,,,39.66
1867254,Medonho´s F.C.,0.00,,,,,,,,,,,,,,,,,,,55.96


In [334]:
# ================================
# Debug: escalacao nominal e pontuacoes (1 time)
# ================================

# Defina manualmente para sobrescrever a escolha automatica
# Exemplo: time_id_debug = 1488983

time_id_debug = 1326835
if time_id_debug is None:
    try:
        if hasattr(df_pontuacoes, "index") and len(df_pontuacoes.index) > 0:
            time_id_debug = int(list(df_pontuacoes.index)[0])
    except Exception:
        time_id_debug = None

    if time_id_debug is None:
        try:
            if isinstance(ids_participantes, list) and len(ids_participantes) > 0:
                time_id_debug = int(ids_participantes[0])
        except Exception:
            time_id_debug = None

if time_id_debug is not None:
    payload = fetch_time_payload(time_id_debug, rodada_atual)
    atletas = payload.get("atletas", []) if isinstance(payload, dict) else []
    reservas = payload.get("reservas", []) if isinstance(payload, dict) else []
    luxo_id = payload.get("reserva_luxo_id") if isinstance(payload, dict) else None

    def _nome_atleta(a):
        if not isinstance(a, dict):
            return "?"
        return a.get("apelido") or a.get("nome") or a.get("nome_cartola") or str(a.get("atleta_id"))

    def _pts(a):
        try:
            return float(mapa_pontuados.get(int(a.get("atleta_id")), 0.0))
        except Exception:
            return 0.0

    def _setor(pos_id):
        return setor_por_posicao(pos_id)

    ordem_setores = ["Goleiro", "Laterais", "Zagueiros", "Meias", "Atacantes", "Tecnico"]

    def _ordenar_por_setor(lista):
        def key(a):
            return ordem_setores.index(_setor(a.get("posicao_id"))) if _setor(a.get("posicao_id")) in ordem_setores else 999
        return sorted([a for a in lista if isinstance(a, dict)], key=key)

    atletas_ordenados = _ordenar_por_setor(atletas)
    reservas_ordenadas = _ordenar_por_setor(reservas)

    print("DEBUG ESCALACAO - time_id:", time_id_debug, "rodada:", rodada_atual)
    print("RESERVA LUXO ID:", luxo_id)

    if atletas_ordenados:
        print("TITULARES (inclui tecnico):")
        setor_atual = None
        for a in atletas_ordenados:
            setor = _setor(a.get("posicao_id"))
            if setor != setor_atual:
                setor_atual = setor
                print(f"  [{setor}]")
            print(
                "   -",
                _nome_atleta(a),
                "id:", a.get("atleta_id"),
                "pos:", a.get("posicao_id"),
                "entrou:", a.get("entrou_em_campo"),
                "pts:", _pts(a)
            )
    else:
        print("TITULARES: (vazio)")

    if reservas_ordenadas:
        print("RESERVAS:")
        setor_atual = None
        for r in reservas_ordenadas:
            setor = _setor(r.get("posicao_id"))
            if setor != setor_atual:
                setor_atual = setor
                print(f"  [{setor}]")
            print(
                "   -",
                _nome_atleta(r),
                "id:", r.get("atleta_id"),
                "pos:", r.get("posicao_id"),
                "entrou:", r.get("entrou_em_campo"),
                "pts:", _pts(r)
            )
    else:
        print("RESERVAS: (vazio)")


DEBUG ESCALACAO - time_id: 1326835 rodada: 1
RESERVA LUXO ID: 92696
TITULARES (inclui tecnico):
  [Goleiro]
   - Léo Linck id: 107183 pos: 1 entrou: False pts: 0.0
  [Laterais]
   - Ramon id: 101597 pos: 2 entrou: False pts: 4.7
   - Igor Formiga id: 106708 pos: 2 entrou: False pts: 0.0
  [Zagueiros]
   - Willian Machado id: 101908 pos: 3 entrou: False pts: 0.0
   - André Ramalho id: 92440 pos: 3 entrou: False pts: 0.0
  [Meias]
   - Giovanni Augusto id: 68938 pos: 4 entrou: False pts: 1.7
   - Montoro id: 142890 pos: 4 entrou: False pts: 0.0
   - Eduardo id: 68708 pos: 4 entrou: False pts: 0.0
  [Atacantes]
   - Pedro Rocha id: 90302 pos: 5 entrou: False pts: 2.8
   - Arthur Cabral id: 92171 pos: 5 entrou: False pts: 0.0
   - Amuzu id: 142004 pos: 5 entrou: False pts: 1.7
  [Tecnico]
   - Rafael Guanaes id: 104027 pos: 6 entrou: False pts: 0.0
RESERVAS:
  [Goleiro]
   - Gabriel id: 70449 pos: 1 entrou: False pts: 6.8
  [Laterais]
   - Walter Clar id: 105664 pos: 2 entrou: False pts: 9

In [335]:
if False:  # debug desativado
    # ================================
    # Inspecao JSON do time (reservas/banco/luxo)
    # ================================
    
    time_id_teste = None
    try:
        if hasattr(df_pontuacoes, "index") and len(df_pontuacoes.index) > 0:
            time_id_teste = int(list(df_pontuacoes.index)[0])
    except Exception:
        time_id_teste = None
    
    if time_id_teste is None:
        try:
            if isinstance(ids_participantes, list) and len(ids_participantes) > 0:
                time_id_teste = int(ids_participantes[0])
        except Exception:
            time_id_teste = None
    
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id_teste}/{rodada_atual}",
        f"https://api.cartolafc.globo.com/time/{time_id_teste}/{rodada_atual}",
    ]
    
    url_ok = None
    response = None
    data = {}
    
    for url in endpoints:
        try:
            r = requests.get(url, headers=HEADERS, timeout=15)
            if r.status_code == 200:
                url_ok = url
                response = r
                data = r.json()
                break
        except Exception:
            continue
    
    if response is None:
        # tenta o ultimo endpoint mesmo assim para debug
        try:
            r = requests.get(endpoints[-1], headers=HEADERS, timeout=15)
            url_ok = endpoints[-1]
            response = r
            data = r.json()
        except Exception:
            data = {}
    
    print("TESTE time_id:", time_id_teste, "rodada:", rodada_atual)
    print("URL usada:", url_ok)
    print("STATUS:", getattr(response, "status_code", None))
    
    if isinstance(data, dict):
        print("TOP-LEVEL KEYS:", list(data.keys()))
    else:
        print("TOP-LEVEL KEYS:", "(data nao e dict)")
    
    if isinstance(data, dict) and isinstance(data.get("time"), dict):
        print("TIME KEYS:", list(data["time"].keys()))
    
    if isinstance(data, dict) and "atletas" in data:
        atletas = data.get("atletas")
        try:
            print("ATLETAS type:", type(atletas), "len:", len(atletas))
        except Exception:
            print("ATLETAS type:", type(atletas), "len:", "?")
        if isinstance(atletas, list) and len(atletas) > 0 and isinstance(atletas[0], dict):
            print("ATLETA[0] KEYS:", list(atletas[0].keys()))
    
    
    def find_keys(obj, needles=("reserv", "banco", "supl", "luxo"), path="$"):
        hits = []
        try:
            if isinstance(obj, dict):
                for k, v in obj.items():
                    try:
                        ks = str(k).lower()
                    except Exception:
                        ks = ""
                    if any(n in ks for n in needles):
                        hits.append(f"{path}.{k}")
                    hits.extend(find_keys(v, needles, f"{path}.{k}"))
            elif isinstance(obj, list):
                for i, v in enumerate(obj):
                    hits.extend(find_keys(v, needles, f"{path}[{i}]"))
        except Exception:
            pass
        return hits
    
    hits = find_keys(data)
    print("HITS RESERVA/BANCO/LUXO:", hits if hits else "NENHUM")
    
    # Preview simples (se houver hits)
    if hits:
        print("PREVIEW (sub-objetos provaveis):")
        if isinstance(data, dict):
            for k in ("reservas", "banco", "reserva_de_luxo", "suplentes"):
                if k in data:
                    val = data.get(k)
                    if isinstance(val, list):
                        print(k, "list len:", len(val), "first2:", val[:2])
                    elif isinstance(val, dict):
                        print(k, "dict keys:", list(val.keys())[:10])
                    else:
                        print(k, "value:", val)
            if isinstance(data.get("time"), dict):
                t = data["time"]
                for k in ("reservas", "banco", "reserva_de_luxo", "suplentes"):
                    if k in t:
                        val = t.get(k)
                        if isinstance(val, list):
                            print("time." + k, "list len:", len(val), "first2:", val[:2])
                        elif isinstance(val, dict):
                            print("time." + k, "dict keys:", list(val.keys())[:10])
                        else:
                            print("time." + k, "value:", val)


In [336]:
# Exportacao (mantem formato)
excel_path = "Pontuacoes_Por_Rodada_Liga_Eliminacao_20.xlsx"
df_pontuacoes.to_excel(excel_path)
print(f"? Arquivo Excel salvo: {excel_path}")

# JS principal

df_dict = df_pontuacoes.fillna("").to_dict(orient="index")
js_content = f"const pontuacoesPorRodada = {json.dumps(df_dict, indent=2, ensure_ascii=False)};"

# (Opcional) eliminados por rodada
elim_js = ""
try:
    elim_js = f"const eliminadosPorRodada = {json.dumps(eliminados_por_rodada, indent=2, ensure_ascii=False)};"
except Exception:
    elim_js = ""

# Parcial da rodada atual (JS)
try:
    col_parcial = f"Parcial Rodada {rodada_atual}"
    parcial_payload = {
        "rodada": int(rodada_atual),
        "times": {
            str(tid): float(df_pontuacoes.at[tid, col_parcial])
            for tid in df_pontuacoes.index
            if str(df_pontuacoes.at[tid, col_parcial]) not in ("", "nan")
        }
    }
except Exception:
    parcial_payload = {"rodada": int(rodada_atual), "times": {}}

parcial_js = f"const pontuacaoParcialRodadaAtual = {json.dumps(parcial_payload, indent=2, ensure_ascii=False)};"

# Grava JS principal + parcial
js_path = "Pontuacoes_Por_Rodada_Liga_Eliminacao_20.js"
with open(js_path, "w", encoding="utf-8") as f:
    f.write(js_content)
    if elim_js:
        f.write("\n\n")
        f.write(elim_js)
    f.write("\n\n")
    f.write(parcial_js)

print(f"? Arquivo JS salvo: {js_path}")

# (Opcional) eliminados por rodada em arquivo separado
try:
    if elim_js:
        with open("Eliminados_Por_Rodada_Liga_Eliminacao_20.js", "w", encoding="utf-8") as f:
            f.write(elim_js)
        print("? Arquivo JS salvo: Eliminados_Por_Rodada_Liga_Eliminacao_20.js")
except Exception:
    pass


? Arquivo Excel salvo: Pontuacoes_Por_Rodada_Liga_Eliminacao_20.xlsx
? Arquivo JS salvo: Pontuacoes_Por_Rodada_Liga_Eliminacao_20.js
? Arquivo JS salvo: Eliminados_Por_Rodada_Liga_Eliminacao_20.js


C:\Users\ferna\AppData\Local\Temp\ipykernel_16788\1541966960.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dict = df_pontuacoes.fillna("").to_dict(orient="index")


## Pontuacao dos Finalistas/Campeoes da Liga Eliminacao (1? turno)


In [337]:
# Soma das pontuacoes ate a rodada final do turno
rodadas_disponiveis = [col for col in df_pontuacoes.columns if col.startswith("Rodada ")]
rodadas_ate_fim = [col for col in rodadas_disponiveis if int(col.split()[-1]) <= FIM_TURNO]

# Soma apenas colunas numericas (strings vazias viram 0)
df_soma = pd.to_numeric(df_pontuacoes[rodadas_ate_fim], errors="coerce").fillna(0.0).sum(axis=1)

df_classificacao_rodada_final = pd.DataFrame({
    "ID": df_pontuacoes.index,
    "Time": df_pontuacoes["Time"],
    f"Total ate Rodada {FIM_TURNO}": df_soma,
}).sort_values(by=f"Total ate Rodada {FIM_TURNO}", ascending=False).reset_index(drop=True)

print(df_classificacao_rodada_final)


TypeError: arg must be a list, tuple, 1-d array, or Series